In [2]:
# pip install datasets

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/us-patent-phrase-to-phrase-matching/sample_submission.csv
/kaggle/input/us-patent-phrase-to-phrase-matching/train.csv
/kaggle/input/us-patent-phrase-to-phrase-matching/test.csv


In [2]:
data = pd.read_csv('/kaggle/input/us-patent-phrase-to-phrase-matching/train.csv')
data.head()

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00


In [3]:
data.shape

(36473, 5)

In [4]:
data.describe(include='object')

,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,37d61fd2272659b1,component composite coating,composition,H01
freq,1,152,24,2186


In [5]:
data['input'] = 'TEXT1:'+ data.context + 'TEXT2:' + data.target + 'ANC:' + data.anchor
data.input.head(10)

0    TEXT1:A47TEXT2:abatement of pollutionANC:abate...
1           TEXT1:A47TEXT2:act of abatingANC:abatement
2          TEXT1:A47TEXT2:active catalystANC:abatement
3      TEXT1:A47TEXT2:eliminating processANC:abatement
4            TEXT1:A47TEXT2:forest regionANC:abatement
5         TEXT1:A47TEXT2:greenhouse gasesANC:abatement
6           TEXT1:A47TEXT2:increased rateANC:abatement
7        TEXT1:A47TEXT2:measurement levelANC:abatement
8        TEXT1:A47TEXT2:minimising soundsANC:abatement
9    TEXT1:A47TEXT2:mixing core materialsANC:abatement
Name: input, dtype: object

In [6]:
from datasets import Dataset, DatasetDict
df = Dataset.from_pandas(data)
df

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input'],
    num_rows: 36473
})

In [84]:
model = 'microsoft/deberta-v3-small'

In [85]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenz = AutoTokenizer.from_pretrained(model, padding='max_len')

loading configuration file https://huggingface.co/microsoft/deberta-v3-small/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/8e0c12a7672d1d36f647c86e5fc3a911f189d8704e2bc94dde4a1ffe38f648fa.9df96bac06c2c492bc77ad040068f903c93beec14607428f25bf9081644ad0da
Model config DebertaV2Config {
  "_name_or_path": "microsoft/deberta-v3-small",
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 768,
  "pos_att_type": [
    "p2c",
    "c2p"
  ],
  "position_biased_input": false,
  "position_buckets": 256,
  "relative_attention": true,
  "share_att

In [86]:
tokenz.tokenize('Hi! I am really excited about the $uperman: Legacy film for the DCU')

['▁Hi',
 '!',
 '▁I',
 '▁am',
 '▁really',
 '▁excited',
 '▁about',
 '▁the',
 '▁$',
 'up',
 'erman',
 ':',
 '▁Legacy',
 '▁film',
 '▁for',
 '▁the',
 '▁DCU']

In [87]:
tokenz.tokenize('Okay, but will Ras Al Ghul or Slade Wilson conjure up in Brave and the Bold????')

['▁Okay',
 ',',
 '▁but',
 '▁will',
 '▁Ras',
 '▁Al',
 '▁G',
 'hul',
 '▁or',
 '▁Slade',
 '▁Wilson',
 '▁conjure',
 '▁up',
 '▁in',
 '▁Brave',
 '▁and',
 '▁the',
 '▁Bold',
 '?',
 '?',
 '?',
 '?']

In [88]:
def tokenize(x): 
    return tokenz(x['input'])

In [89]:
tok_df = df.map(tokenize, batched=True)

  0%|          | 0/37 [00:00<?, ?ba/s]

In [90]:
row = tok_df[0]
row['input'], row['input_ids']

('TEXT1:A47TEXT2:abatement of pollutionANC:abatement',
 [1,
  54453,
  435,
  294,
  558,
  5753,
  104917,
  445,
  294,
  16191,
  297,
  17019,
  265,
  6435,
  64097,
  294,
  16191,
  297,
  17019,
  2])

In [91]:
tok_df = tok_df.rename_columns({'score':'labels'})

In [92]:
test_df = pd.read_csv('/kaggle/input/us-patent-phrase-to-phrase-matching/test.csv')
test_df.describe()

,id,anchor,target,context
count,36,36,36,36
unique,36,34,36,29
top,4112d61851461f60,el display,inorganic photoconductor drum,G02
freq,1,2,1,3


In [93]:
dds = tok_df.train_test_split(0.25, seed=42)
dds

DatasetDict({
    train: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 27354
    })
    test: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9119
    })
})

In [94]:
test_df['input'] = 'TEXT1:'+ test_df.context + 'TEXT2:' + test_df.target + 'ANC:' + test_df.anchor
test_df = Dataset.from_pandas(test_df).map(tokenize, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [95]:
def corr(x,y):
    return np.corrcoef(x,y)[0][1]

In [96]:
def corr_d(test_df):
    return {'pearson': corr(*test_df)}

In [97]:
# pip install transformers

In [98]:
from transformers import Trainer, TrainingArguments
bs = 128
epochs=4
lr = 7e-5

In [99]:
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1,
                         lr_scheduler_type='cosine', fp16 = True,
                         evaluation_strategy='epoch', per_device_train_batch_size=bs,
                         per_device_eval_batch_size=bs*2, num_train_epochs=epochs,
                         weight_decay=0.01, report_to='none'
                        )



PyTorch: setting up devices


In [102]:
model_x = AutoModelForSequenceClassification.from_pretrained(model, num_labels=1)
trainer = Trainer(model_x, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                 tokenizer=tokenz, compute_metrics=corr_d)

loading configuration file https://huggingface.co/microsoft/deberta-v3-small/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/8e0c12a7672d1d36f647c86e5fc3a911f189d8704e2bc94dde4a1ffe38f648fa.9df96bac06c2c492bc77ad040068f903c93beec14607428f25bf9081644ad0da
Model config DebertaV2Config {
  "_name_or_path": "microsoft/deberta-v3-small",
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 768,
  "pos_att_type": [
    "p2c",
    "c2p"
  ],
  "position_biased_input"

In [103]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: anchor, target, id, input, context. If anchor, target, id, input, context are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 27354
  Num Epochs = 4
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 1
  Total optimization steps = 428
/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.030082,0.756549
2,No log,0.026892,0.799371
3,No log,0.027595,0.806051
4,No log,0.025550,0.811190


The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: anchor, target, id, input, context. If anchor, target, id, input, context are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9119
  Batch size = 512
The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: anchor, target, id, input, context. If anchor, target, id, input, context are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9119
  Batch size = 512
The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: anchor, target, id, input, context. If anchor, ta

TrainOutput(global_step=428, training_loss=0.03395047811704261, metrics={'train_runtime': 172.5605, 'train_samples_per_second': 634.073, 'train_steps_per_second': 2.48, 'total_flos': 676715015824740.0, 'train_loss': 0.03395047811704261, 'epoch': 4.0})

In [104]:
preds = trainer.predict(test_df).predictions.astype('float')
preds = np.clip(preds, 0,1)
preds

The following columns in the test set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: anchor, target, id, input, context. If anchor, target, id, input, context are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 36
  Batch size = 512


array([[0.57861328],
       [0.77734375],
       [0.50976562],
       [0.42016602],
       [0.10113525],
       [0.64013672],
       [0.54296875],
       [0.        ],
       [0.28515625],
       [1.        ],
       [0.37597656],
       [0.30175781],
       [0.71142578],
       [0.68359375],
       [0.80078125],
       [0.53466797],
       [0.29663086],
       [0.        ],
       [0.64892578],
       [0.36474609],
       [0.32275391],
       [0.17651367],
       [0.03182983],
       [0.27050781],
       [0.59619141],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.70214844],
       [0.33349609],
       [0.00930786],
       [0.73388672],
       [0.54345703],
       [0.46557617],
       [0.20336914]])